# Zipline Backtesting

#### COPYRIGHT QUANT SCIENCE UNIVERSITY
#### Goal: Get you started making progress with backtesting
****

## ENVIRONMENT INSTRUCTIONS WITH CONDA:

Use these command line scripts to create a zipline_backtesting environment with the necessary python libraries to complete this tutorial.

``` bash
conda create -n zipline_backtesting
conda activate zipline_backtesting
conda install python
pip install zipline-reloaded numpy==1.26.4 plotly nbformat
```

In [ ]:
# IMPORTS
from zipline.api import order_target, record, symbol
from zipline import run_algorithm
from zipline.data import bundles

import pandas as pd
import plotly.express as px
import os

import warnings

# Suppress specific FutureWarning messages
warnings.filterwarnings("ignore", category=FutureWarning, message=".*fillna.*")
warnings.filterwarnings("ignore", category=FutureWarning, message=".*Series.__setitem__.*")

## STEP 1: INGEST A DATA BUNDLE

* ONLY NEED TO RUN THIS ONCE.
* THIS BUILDS THE QUANDL FREE DATA UP TO 2018. 
* Instructions: https://docs.data.nasdaq.com/v1.0/docs/getting-started

In [ ]:
os.environ["QUANDL_API_KEY"] = "YOUR_API_KEY"
bundle = "quandl"
bundles.ingest(bundle)

## STEP 2: CREATE A TRADING STRATEGY

In [ ]:
def initialize(context):
    # Define the stock to trade
    context.asset = symbol('AAPL')
    
    # Set the historical windows
    context.short_window = 10
    context.long_window = 30

def handle_data(context, data):
    # Get historical data
    short_mavg = data.history(context.asset, 'price', context.short_window, '1d').mean()
    long_mavg = data.history(context.asset, 'price', context.long_window, '1d').mean()

    # Trading logic
    if short_mavg > long_mavg:
        # Buy signal
        order_target(context.asset, 100)
    else:
        # Sell signal
        order_target(context.asset, 0)
    
    # Record the moving averages for later analysis
    record(
        short_mavg=short_mavg, 
        long_mavg=long_mavg, 
        price=data.current(context.asset, 'price')
    )

## STEP 3: MAKE THE BASIC BACKTEST

In [ ]:
# Define the backtest parameters
start = pd.Timestamp('2016-01-01')
end = pd.Timestamp('2017-12-31')
capital_base = 10000

# Run the algorithm
result = run_algorithm(
    start=start, 
    end=end, 
    initialize=initialize,
    handle_data=handle_data, 
    capital_base=capital_base,
    bundle='quandl'
)

In [ ]:
result

,period_open,period_close,long_exposure,short_exposure,pnl,gross_leverage,capital_used,net_leverage,transactions,positions,...,max_drawdown,max_leverage,excess_return,treasury_period_return,benchmark_period_return,benchmark_volatility,trading_days,period_label,algorithm_period_return,algo_volatility
2016-01-04 21:00:00+00:00,2016-01-04 14:31:00+00:00,2016-01-04 21:00:00+00:00,0.0,0.0,0.0,0.000000,0.0,0.000000,[],[],...,0.000000,0.000000,0.0,0.0,0.0,NaN,1,2016-01,0.000000,NaN
2016-01-05 21:00:00+00:00,2016-01-05 14:31:00+00:00,2016-01-05 21:00:00+00:00,0.0,0.0,0.0,0.000000,0.0,0.000000,[],[],...,0.000000,0.000000,0.0,0.0,0.0,0.0,2,2016-01,0.000000,0.000000
2016-01-06 21:00:00+00:00,2016-01-06 14:31:00+00:00,2016-01-06 21:00:00+00:00,0.0,0.0,0.0,0.000000,0.0,0.000000,[],[],...,0.000000,0.000000,0.0,0.0,0.0,0.0,3,2016-01,0.000000,0.000000
2016-01-07 21:00:00+00:00,2016-01-07 14:31:00+00:00,2016-01-07 21:00:00+00:00,0.0,0.0,0.0,0.000000,0.0,0.000000,[],[],...,0.000000,0.000000,0.0,0.0,0.0,0.0,4,2016-01,0.000000,0.000000
2016-01-08 21:00:00+00:00,2016-01-08 14:31:00+00:00,2016-01-08 21:00:00+00:00,0.0,0.0,0.0,0.000000,0.0,0.000000,[],[],...,0.000000,0.000000,0.0,0.0,0.0,0.0,5,2016-01,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2017-12-22 21:00:00+00:00,2017-12-22 14:31:00+00:00,2017-12-22 21:00:00+00:00,17501.0,0.0,0.0,1.168765,0.0,1.168765,[],"[{'sid': Equity(8 [AAPL]), 'amount': 100, 'cos...",...,-0.196763,1.190751,0.0,0.0,0.0,0.0,499,2017-12,0.497393,0.172527
2017-12-26 21:00:00+00:00,2017-12-26 14:31:00+00:00,2017-12-26 21:00:00+00:00,17057.0,0.0,-444.0,1.173922,0.0,1.173922,[],"[{'sid': Equity(8 [AAPL]), 'amount': 100, 'cos...",...,-0.196763,1.190751,0.0,0.0,0.0,0.0,500,2017-12,0.452993,0.173711
2017-12-27 21:00:00+00:00,2017-12-27 14:31:00+00:00,2017-12-27 21:00:00+00:00,17060.0,0.0,3.0,1.173886,0.0,1.173886,[],"[{'sid': Equity(8 [AAPL]), 'amount': 100, 'cos...",...,-0.196763,1.190751,0.0,0.0,0.0,0.0,501,2017-12,0.453293,0.173537
2017-12-28 21:00:00+00:00,2017-12-28 14:31:00+00:00,2017-12-28 21:00:00+00:00,17108.0,0.0,48.0,1.173313,0.0,1.173313,[],"[{'sid': Equity(8 [AAPL]), 'amount': 100, 'cos...",...,-0.196763,1.190751,0.0,0.0,0.0,0.0,502,2017-12,0.458093,0.173373


## STEP 4: PLOT THE PERFORMANCE

In [ ]:
# Plot the performance
px.line(
    result.portfolio_value, 
    title = 'Portfolio Value Over Time',
    template='simple_white'
)

![Portfolio Performance](img/portfolio_performance.png)